In [3]:
import pandas as pd 

data=pd.read_csv('../data/diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
import json
from datetime import datetime

# Dictionnaire des correspondances FHIR pour chaque colonne
observation_metadata = {
    "Pregnancies": {"code": "num-pregnancies", "display": "Number of Pregnancies", "unit": "", "system": "", "code_unit": ""},
    "Glucose": {"code": "2339-0", "display": "Glucose [mg/dL]", "unit": "mg/dL", "system": "http://unitsofmeasure.org", "code_unit": "mg/dL"},
    "BloodPressure": {"code": "8462-4", "display": "Diastolic blood pressure", "unit": "mmHg", "system": "http://unitsofmeasure.org", "code_unit": "mm[Hg]"},
    "SkinThickness": {"code": "56115-9", "display": "Skinfold thickness", "unit": "mm", "system": "http://unitsofmeasure.org", "code_unit": "mm"},
    "Insulin": {"code": "2075-0", "display": "Insulin", "unit": "µU/mL", "system": "http://unitsofmeasure.org", "code_unit": "uU/mL"},
    "BMI": {"code": "39156-5", "display": "Body Mass Index", "unit": "kg/m2", "system": "http://unitsofmeasure.org", "code_unit": "kg/m2"},
    "DiabetesPedigreeFunction": {"code": "pedigree-func", "display": "Diabetes Pedigree Function", "unit": "", "system": "", "code_unit": ""},
    "Age": {"code": "age", "display": "Age", "unit": "years", "system": "http://unitsofmeasure.org", "code_unit": "a"},
    "Outcome": {"code": "diabetes-status", "display": "Diabetes Diagnosis", "unit": "", "system": "", "code_unit": ""}
}

fhir_data = []


for idx, row in data.iterrows():
    patient_id = f"patient-{idx+1}"

    
    patient_resource = {
        "resourceType": "Patient",
        "id": patient_id
    }
    fhir_data.append(patient_resource)

    # Créer les Observations
    for field in data.columns:
        value = row[field]
        meta = observation_metadata[field]

        observation = {
            "resourceType": "Observation",
            "id": f"obs-{field.lower()}-{idx+1}",
            "status": "final",
            "code": {
                "coding": [{
                    "system": meta["system"],
                    "code": meta["code"],
                    "display": meta["display"]
                }],
                "text": meta["display"]
            },
            "subject": {
                "reference": f"Patient/{patient_id}"
            },
            "effectiveDateTime": datetime.today().strftime("%Y-%m-%d"),
        }

        # Ajouter la valeur
        if meta["unit"]:  # valeur numérique avec unité
            observation["valueQuantity"] = {
                "value": float(value),
                "unit": meta["unit"],
                "system": meta["system"],
                "code": meta["code_unit"]
            }
        else:  # valeur sans unité
            observation["valueInteger"] = int(value) if field in ["Pregnancies", "Age", "Outcome"] else float(value)

        fhir_data.append(observation)

# Exporter en JSON
with open("../data/fhir_diabetes_data.json", "w") as f:
    json.dump(fhir_data, f, indent=2)

print("✅ Données FHIR générées avec succès dans 'fhir_diabetes_data.json'")


✅ Données FHIR générées avec succès dans 'fhir_diabetes_data.json'
